In [1]:
!pip install mxnet

You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.


# Vortrainiertes Modell

Hier verwenden wir ein bereits trainiertes, komplexes Convolutional Neural Network.

Das Netzwerk wurde auf dem CIFAR-10 Datensatz trainiert.

Es geht hier mehrheitlich darum, aufzuzeigen, wozu grosse CNNs bei **grossen Bilddatensätzen** in der Lage sind.

In [2]:
import numpy as np

from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

import pandas as pd
import pickle

In [3]:
def plot_confusion_matrix(y_true: any, y_pred: any):
    """
        Takes true and predicted labels (arrays) as input. Based on this, calculates and plots a confusion matrix.
    """
    labels = np.unique(y_true)
    fig = plt.figure(figsize=(len(labels), len(labels)))
    ConfusionMatrixDisplay(
      confusion_matrix=confusion_matrix(y_true=y_true, y_pred=y_pred, labels=labels, normalize='all'),
      display_labels=labels
    ).plot(ax=fig.gca(), cmap="BuPu", xticks_rotation='vertical', include_values=True)
    plt.show()

In [4]:
!pip install tensorflow
!pip install gluoncv

You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/beni/Documents/Kennwerte/IdeaProjects/cas_machine_learning_cifar10/venv/bin/python -m pip install --upgrade pip' command.


In [5]:
# Class mapping from pretrained model
class_names = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

## ResNet110 - Vortrainiertes Modell (auf CIFAR 10)

Wir laden das bereits trainierte Netzwerk mittels der `mxnet` Library.

Wir müssen das Modell **nicht mehr trainieren** und laden es hier nur.

Das verwendete Modell ist eine Implementierung von diesem Paper: https://arxiv.org/abs/1603.05027

In [6]:
from mxnet.gluon.data.vision import transforms

import mxnet as mx
from gluoncv.model_zoo import get_model

ctx = [mx.cpu(0)]

# Get the model CIFAR_ResNet20_v1, with 10 output classes, without pre-trained weights
net = get_model('cifar_resnet110_v2', pretrained=True, classes=10)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])  # Normalization ist analog wie bei Training des Netwerkes
])

# Anwendung des Modells auf dem Test Set

Wenn wir mit der Leistung unseres Modells auf dem `Validation Set` zufrieden sind, können wir es auf das `Test Set` anwenden.

In [7]:
# Daten laden und aufteilen in Features und Labels
with open('../data/test.pkl', 'rb') as f:
    data_test = pickle.load(f)
X_test = data_test["images"]
y_test = data_test["labels"]

In [8]:
%time

y_test_pred_prob = net(transform_test(mx.ndarray.array(X_test)))

y_test_pred = np.argmax(y_test_pred_prob.asnumpy(), axis=1)
# Ziffern zurück zu Labels transformieren (die Reihenfolge ist vom Netz vorgegeben)
y_test_pred_enc = np.vectorize(class_names.get)(y_test_pred)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.91 µs


Evaluieren wir unsere test-set predictions

In [9]:
accuracy_score(y_true=y_test, y_pred=y_test_pred_enc)

0.9554